In [19]:
import os 
os.chdir("../")

In [20]:
%pwd

'C:\\FOLDER D ASLI\\Kuliah\\Semester 7\\Bismillah Skripsi\\Chatbot RAG\\rag-chatbot-tourism\\resources'

In [1]:
import sys, platform
print("Python exe:", sys.executable)
print("Python ver:", platform.python_version())

Python exe: C:\Users\akuns\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe
Python ver: 3.13.9


In [2]:
import langchain
import pypdf

from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✅ LangChain berhasil diimport!")
print("✅ PyPDF berhasil diimport!")

✅ LangChain berhasil diimport!
✅ PyPDF berhasil diimport!


In [3]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader

def load_pdf_files(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents
    print("data berhasil di load")

In [4]:
extracted_data = load_pdf_files("data")

In [5]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-10-29T11:39:01+07:00', 'author': 'Amar Al Farizi', 'moddate': '2025-10-29T11:39:01+07:00', 'source': 'data\\data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa kera

In [6]:
len (extracted_data)

3

In [7]:
from langchain_core.documents import Document
from typing import List

def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """Simpan hanya page_content dan metadata 'source'."""
    minimal = []
    for d in docs:
        minimal.append(
            Document(
                page_content=d.page_content,
                metadata={"source": d.metadata.get("source")}
            )
        )
    return minimal

# === pakai fungsi & CEK hasilnya ===
min_docs = filter_to_minimal_docs(extracted_data)

# cek 1 dokumen
print(min_docs[0].metadata)          # -> {'source': 'data\\data.pdf'}
# pastikan tidak ada kunci lain:
print(list(min_docs[0].metadata.keys()))



{'source': 'data\\data.pdf'}
['source']


In [8]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [9]:
extracted_data

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-10-29T11:39:01+07:00', 'author': 'Amar Al Farizi', 'moddate': '2025-10-29T11:39:01+07:00', 'source': 'data\\data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.  \n2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa kera

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [11]:
texts_chunk = text_split(minimal_docs)
print(f"Number of chunks: {len(texts_chunk)}")

Number of chunks: 20


In [12]:
texts_chunk

[Document(metadata={'source': 'data\\data.pdf'}, page_content='Wisata Alam Banyumas \n1. Lokawisata Baturraden  terletak di Karangmangu, Baturraden . Lokawisata \nBaturraden merupakan destinasi utama di kaki Gunung Slamet yang menawarkan \ntaman rekreasi keluarga, kolam air panas, dan area bermain anak. Udara sejuk \npegunungan menjadikannya tempat ideal untuk berlibur bersama keluarga. Harga \ntiket dewasa R p20.000 (weekday), anak -anak Rp10.000 (weekday), dan weekend \nRp25.000 (dewasa), Rp12.500 (anak). Jam buka 7 AM dan jam tutup 7 PM.'),
 Document(metadata={'source': 'data\\data.pdf'}, page_content='2. Telaga Sunyi berlokasi di Ketenger, Baturraden. Telaga Sunyi adalah kolam alami \ndengan air jernih di tengah hutan pinus. Suasananya tenang dan sejuk, cocok untuk \nbersantai dan menikmati keindahan alam tanpa keramaian. Harga tiket dewasa \n15000, anak-anak Tidak Ada, weekend Tidak Ada. Jam buka 8 AM dan jam tutup 4 PM. \n3. Hutan Pinus Limpakuwus berlokasi di Jl Baturraden Timur